In [1]:
import pandas as pd
import json
import numpy as np
from itertools import chain
from collections import defaultdict

# Participation details

Here we show some basic information about participants in the study.

In [2]:
df = pd.read_csv("pre_study_participation.csv")

In [3]:
age_map = {
     0: "0-15",
    16: "16-20",
    21: "21-30",
    29: "31-40",
    41: "41-50",
    51: "51-65",
    65: "65+"
}

gender_map = {
     0: "male",
     1: "female",
     2: "other"
}

## Age groups

In [4]:
df["age_group_label"] = df["age_group"].replace(age_map)

# Preserve the original bin order (so 0‑15 appears first, 65+ last)
cat_order = list(age_map.values())
df["age_group_label"] = pd.Categorical(df["age_group_label"], ordered=True, categories=cat_order)

age_counts = df["age_group_label"].value_counts().sort_index()
age_counts.to_frame("count")

,count
age_group_label,
0-15,0
16-20,4
21-30,12
31-40,5
41-50,2
51-65,1
65+,0


## Gender counts

In [5]:
df["gender_label"] = df["gender"].replace(gender_map)
gender_counts = df["gender_label"].value_counts().reindex(gender_map.values(), fill_value=0)
gender_counts.to_frame("count")

,count
gender_label,
male,17
female,6
other,1


## Average completion time

In [6]:
df["time_joined"]   = pd.to_datetime(df["time_joined"])
df["time_finished"] = pd.to_datetime(df["time_finished"])

durations = df["time_finished"] - df["time_joined"]
mean_dur  = durations.mean()

# Convert to mm:ss
mean_minutes, mean_seconds = divmod(int(mean_dur.total_seconds()), 60)
avg_time_str = f"{mean_minutes}:{mean_seconds:02d}"

print(f"Average study duration: {avg_time_str} (mm:ss)")

Average study duration: 9:13 (mm:ss)


# User study interactions

Now we will analyze user feedback and interactions from the diversity perception phase and recommendation phase of the study.



## Diversity perception phase

`diversity-perception-ended` interaction_type

Data:

```json

{
  "sim_plot": [
    {
      "rating": integer,
      "genre_sim": number,
      "plot_sim": number 
    }
    // ... 
  ],
  "sim_genres": [
      { 
      "rating": integer,
      "genre_sim": number,
      "plot_sim": number
    }
    // ... 
  ],
  "attention_check": [ 
      {
      "rating": integer,
      "genre_sim": number, 
      "plot_sim": number
    }
    // ... 
  ],
}


```

In [7]:
df_int = pd.read_csv("pre_study_interaction.csv")

div_df = df_int[df_int["interaction_type"] == "diversity-perception-ended"].copy()

# If a participant somehow produced more than one such row we keep last one
div_df.sort_values("time", inplace=True)
div_df = div_df.drop_duplicates("participation", keep="last")

In [8]:
def extract_avg_ratings(json_blob):
    d = json.loads(json_blob)
    return {
        "sim_plot_avg": np.mean([item["rating"] for item in d["sim_plot"]]),
        "sim_genres_avg": np.mean([item["rating"] for item in d["sim_genres"]]),
        "attention_check_avg":np.mean([item["rating"] for item in d["attention_check"]]),
    }

ratings_expanded = div_df["data"].apply(extract_avg_ratings).apply(pd.Series)

In [9]:
per_participant = pd.concat(
    [div_df[["participation"]].reset_index(drop=True), ratings_expanded],
    axis=1
)

per_participant

,participation,sim_plot_avg,sim_genres_avg,attention_check_avg
0,1.0,3.666667,3.333333,1.0
1,2.0,NaN,NaN,NaN
2,3.0,NaN,NaN,NaN
3,4.0,NaN,NaN,NaN
4,5.0,NaN,NaN,NaN
5,6.0,NaN,NaN,NaN
6,7.0,NaN,NaN,NaN
7,8.0,NaN,NaN,NaN
8,9.0,NaN,NaN,NaN
9,10.0,NaN,NaN,NaN


In [10]:
overall_avg = per_participant.mean(numeric_only=True).to_frame(name="overall_avg").T
overall_avg

,participation,sim_plot_avg,sim_genres_avg,attention_check_avg
overall_avg,12.5,3.625,3.513889,1.041667


## Recommendation phase

First, we will process for each user, for every iteration, for every algorithm, how many of the recommended movies were selected by the user.  
Then, we will calculate per-user totals and averages of selected movies accross all iterations.  
In the end, we will calculate overall totals and averages for each algorithm.  



In [11]:
df = pd.read_csv("pre_study_interaction.csv")

# Work only with rows relevant to the recommendation phase
phase_mask = df["interaction_type"].isin(["iteration-started", "iteration-ended"])
phase_df = df[phase_mask].copy()

phase_df["data_json"] = phase_df["data"].apply(json.loads)

# Extract the iteration number
phase_df["iteration_no"] = phase_df["data_json"].apply(lambda d: d.get("iteration"))

In [12]:
starts_df = phase_df[phase_df["interaction_type"] == "iteration-started"]
ends_df = phase_df[phase_df["interaction_type"] == "iteration-ended"]

# Keep only necessary columns
starts_df = starts_df[["participation", "iteration_no", "data_json"]]
ends_df = ends_df[["participation", "iteration_no", "data_json"]]

In [13]:
starts_dict = defaultdict(dict)
for _, row in starts_df.iterrows():
    it = row["iteration_no"]
    shown = row["data_json"]["shown"]
    for algo, shown_lists in shown.items():
        if it not in starts_dict[row["participation"]]:
            starts_dict[row["participation"]][it] = {}
        starts_dict[row["participation"]][it][algo] = shown_lists[-1] # for each algo we have a list of shown movies for each iteration (also previos ones) so we get the last one

ends_dict = defaultdict(dict)
for _, row in ends_df.iterrows():
    it = row["iteration_no"]
    data = row["data_json"]
    ratings_obj = data["ratings"][-1]

    ends_dict[row["participation"]][it] = {
        "selected": data["selected"][-1], # iteration-ended contains also previous iteration data, get the last one
        "ratings": ratings_obj
    }


In [14]:
valid_participants = [
    p for p in starts_dict
    if p in ends_dict and starts_dict[p].keys() == ends_dict[p].keys()
]

print(f"Keeping {len(valid_participants)} valid participants.")

Keeping 24 valid participants.


In [15]:
iteration_data = []

for p in valid_participants:
    for it in sorted(starts_dict[p]):
        shown_dict = starts_dict[p][it] # {algo: [id, id...], algo2: [id, id...]}
        selected = set(ends_dict[p][it]["selected"])   # movie_ids the user picked
        ratings = ends_dict[p][it]["ratings"] # {algo: {relevance, diversity}}

        for algo, shown_movies in shown_dict.items():
            sel_for_algo = [m for m in shown_movies if m in selected]

            rating = ratings.get(algo)

            iteration_data.append({
                "participant": p,
                "iteration": it,
                "algo": algo,
                "selected_count": len(sel_for_algo),
                "selected_movies": sel_for_algo,
                "relevance_rating": rating["relevance"],
                "diversity_rating": rating["diversity"]
            })

per_iter_df = pd.DataFrame(iteration_data)
per_iter_df.head(9)

,participant,iteration,algo,selected_count,selected_movies,relevance_rating,diversity_rating
0,1,1,EASE,6,"[6822, 5411, 3585, 6823, 7712, 6824]",5,2
1,1,1,ProfilingRR,5,"[7927, 3585, 7712, 4695, 9430]",4,4
2,1,1,ProfilingLLM,3,"[6411, 7927, 5112]",2,5
3,1,2,EASE,6,"[6813, 8610, 6826, 2675, 6394, 5785]",5,2
4,1,2,ProfilingRR,6,"[4457, 7850, 1375, 8610, 7568, 6825]",4,4
5,1,2,ProfilingLLM,3,"[2822, 8151, 4870]",2,5
6,1,3,EASE,0,[],5,2
7,1,3,ProfilingRR,0,[],4,4
8,1,3,ProfilingLLM,0,[],2,5


In [16]:
def union_lists(list_of_lists):
    return sorted({m for sub in list_of_lists for m in sub})

user_totals = (
    per_iter_df
      .groupby(["participant", "algo"])
      .agg(
          total_selected_count = ("selected_count", "sum"),
          avg_selected_per_iter = ("selected_count", "mean"),
          total_selected_movies = ("selected_movies", union_lists),
          avg_relevance_rating = ("relevance_rating", "mean"),
          avg_diversity_rating = ("diversity_rating", "mean")
      )
      .reset_index()
)

user_totals.head()

,participant,algo,total_selected_count,avg_selected_per_iter,total_selected_movies,avg_relevance_rating,avg_diversity_rating
0,1,EASE,12,4.000000,"[2675, 3585, 5411, 5785, 6394, 6813, 6822, 682...",5.0,2.000000
1,1,ProfilingLLM,6,2.000000,"[2822, 4870, 5112, 6411, 7927, 8151]",2.0,5.000000
2,1,ProfilingRR,11,3.666667,"[1375, 3585, 4457, 4695, 6825, 7568, 7712, 785...",4.0,4.000000
3,2,EASE,11,3.666667,"[0, 177, 190, 1576, 2184, 3675, 4628, 4837, 51...",5.0,3.333333
4,2,ProfilingLLM,7,2.333333,"[41, 570, 2095, 4773, 5354, 6817, 7568]",3.0,3.666667


In [17]:
overall_totals = (
    per_iter_df
      .groupby("algo")
      .agg(
          total_selected_across_users = ("selected_count", "sum"),
          avg_selected_per_user = ("selected_count", "mean"),
          avg_relevance_rating = ("relevance_rating", "mean"),
          avg_diversity_rating = ("diversity_rating", "mean")
      )
      .reset_index()
)

overall_totals


,algo,total_selected_across_users,avg_selected_per_user,avg_relevance_rating,avg_diversity_rating
0,EASE,217,3.013889,4.527778,2.027778
1,ProfilingLLM,94,1.305556,2.388889,3.944444
2,ProfilingRR,164,2.277778,3.638889,2.888889


In [18]:
overall_totals_sorted_selected = (
    overall_totals[["algo", "avg_selected_per_user"]]
    .sort_values("avg_selected_per_user", ascending=False)
    .reset_index(drop=True)
)

overall_totals_sorted_selected["rank"] = overall_totals_sorted_selected.index + 1
overall_totals_sorted_selected

,algo,avg_selected_per_user,rank
0,EASE,3.013889,1
1,ProfilingRR,2.277778,2
2,ProfilingLLM,1.305556,3


In [19]:
overall_totals_sorted_relevance = (
    overall_totals[["algo", "avg_relevance_rating"]]
    .sort_values("avg_relevance_rating", ascending=False)
    .reset_index(drop=True)
)

overall_totals_sorted_relevance["rank"] = overall_totals_sorted_relevance.index + 1
overall_totals_sorted_relevance

,algo,avg_relevance_rating,rank
0,EASE,4.527778,1
1,ProfilingRR,3.638889,2
2,ProfilingLLM,2.388889,3


In [20]:
overall_totals_sorted_diversity = (
    overall_totals[["algo", "avg_diversity_rating"]]
    .sort_values("avg_diversity_rating", ascending=False)
    .reset_index(drop=True)
)

overall_totals_sorted_diversity["rank"] = overall_totals_sorted_diversity.index + 1
overall_totals_sorted_diversity

,algo,avg_diversity_rating,rank
0,ProfilingLLM,3.944444,1
1,ProfilingRR,2.888889,2
2,EASE,2.027778,3
